In [18]:
import json
import parsing
import tiktoken
import pandas as pd

In [19]:
def token_counter(text: str) -> int:
    encoding = tiktoken.encoding_for_model('gpt-4o-mini')
    return len(encoding.encode(text))

In [20]:
def read_str_file(file_path: str) -> str:
    with open(file_path, mode='r', encoding='utf-8') as file:
        return file.read()

## 1. Data Inspection

In [ ]:
file_path = 'marketing_sample_for_trulia_com-real_estate__20190901_20191031__30k_data.csv'

In [22]:
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30002 entries, 0 to 30001
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job Title           30002 non-null  object 
 1   Job Description     30002 non-null  object 
 2   Job Type            0 non-null      float64
 3   Categories          0 non-null      float64
 4   Location            30002 non-null  object 
 5   City                30002 non-null  object 
 6   State               30002 non-null  object 
 7   Country             30002 non-null  object 
 8   Zip Code            16252 non-null  object 
 9   Address             0 non-null      float64
 10  Salary From         0 non-null      float64
 11  Salary To           0 non-null      float64
 12  Salary Period       0 non-null      float64
 13  Apply Url           18392 non-null  object 
 14  Apply Email         0 non-null      float64
 15  Employees           0 non-null      float64
 16  Indu

In [23]:
# check 'Uniq Id' column is unique
'Uniq Id is unique' if df['Uniq Id'].nunique() == df.shape[0] else 'Uniq Id is not unique'

'Uniq Id is unique'

In [24]:
# remove empty columns
df.dropna(axis=1, how='all', inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30002 entries, 0 to 30001
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job Title           30002 non-null  object
 1   Job Description     30002 non-null  object
 2   Location            30002 non-null  object
 3   City                30002 non-null  object
 4   State               30002 non-null  object
 5   Country             30002 non-null  object
 6   Zip Code            16252 non-null  object
 7   Apply Url           18392 non-null  object
 8   Company Name        30000 non-null  object
 9   Employer Logo       18708 non-null  object
 10  Companydescription  19662 non-null  object
 11  Employer Location   3348 non-null   object
 12  Employer City       3316 non-null   object
 13  Employer State      3316 non-null   object
 14  Employer Country    30002 non-null  object
 15  Employer Zip Code   1743 non-null   object
 16  Uniq Id             30

In [26]:
# unique country values
df['Country'].unique()

array(['United States'], dtype=object)

In [27]:
df['State'].value_counts(ascending=True)

State
Oklahoma         1
Home Based       1
Sydney NSW       1
Nevada           1
BC               1
              ... 
IL            1472
FL            1550
NY            2075
TX            2466
CA            4495
Name: count, Length: 81, dtype: int64

## 2. Data Cleaning

In [28]:
# read json file usa_states.json
usa_states = json.loads(read_str_file('usa_states.json'))

In [29]:
# Create a dictionary from usa_states for quick lookup
state_dict = {state['name']: state['abbreviation'] for state in usa_states}

# Function to replace state names with abbreviations or 'N/A'
def fix_state_values(state):
    if state in state_dict:
        return state_dict[state]
    elif state in state_dict.values():
        return state
    else:
        return 'N/A'

# Apply the function to the State column
df['State'] = df['State'].apply(fix_state_values)
df['State'].unique()


array(['CA', 'GA', 'IL', 'MO', 'IA', 'N/A', 'WA', 'NC', 'NY', 'TX', 'ID',
       'FL', 'ME', 'OH', 'VA', 'MD', 'MA', 'NH', 'MI', 'CO', 'NJ', 'OR',
       'OK', 'WY', 'AZ', 'CT', 'WI', 'AR', 'PA', 'SC', 'KY', 'MN', 'IN',
       'NV', 'MT', 'AL', 'UT', 'RI', 'KS', 'TN', 'AK', 'SD', 'NM', 'NE',
       'WV', 'MS', 'VT', 'HI', 'DE', 'LA', 'ND'], dtype=object)

In [30]:
df['Crawl Timestamp'] = pd.to_datetime(df['Crawl Timestamp'], format='%Y-%m-%d %H:%M:%S %z')

In [31]:
df['Crawl Timestamp'].min(), df['Crawl Timestamp'].max()

(Timestamp('2019-08-16 11:27:08+0000', tz='UTC'),
 Timestamp('2019-10-25 23:16:50+0000', tz='UTC'))

## 3. Token analysis

In [32]:
# calculate tokens in 'Job Description'
df['tokens'] = df['Job Description'].apply(token_counter)
df['tokens_markdown'] = df['Job Description'].apply(lambda x: token_counter(parsing.html_to_markdown(x)))

In [33]:
df[['tokens', 'tokens_markdown']].describe(percentiles=[0.75, 0.9, 0.95, 0.99])

,tokens,tokens_markdown
count,30002.000000,30002.000000
mean,978.270349,770.791247
std,419.160302,338.428856
min,62.000000,0.000000
50%,906.000000,717.000000
75%,1189.000000,950.000000
90%,1472.000000,1180.000000
95%,1699.000000,1349.000000
99%,2350.980000,1811.990000
max,6918.000000,4511.000000


In [34]:
df[df['tokens_markdown'] == 0].shape[0]

1

## Saved clean data

In [35]:
df_export = df[df['tokens_markdown'] > 0]
df_export.drop(columns=['tokens', 'tokens_markdown']).to_csv('data_cleaned.csv', index=False)
print(df_export.shape[0], 'rows exported to data_cleaned.csv')

30001 rows exported to data_cleaned.csv
